In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

from splinter import Browser
import time

### Code for getting featured stories from NASA mars mission site


In [2]:
# NASA mars site information
#using request and soup
nasa_url = "https://mars.nasa.gov/news/"

nasa_response = requests.get(nasa_url)

# Create BeautifulSoup object; parse with 'html.parser'
nasa_soup = BeautifulSoup(nasa_response.text, 'lxml')

#print(nasa_soup.prettify())

In [3]:
# results are returned as an iterable list
nasa_results = nasa_soup.find_all('div', class_="slide")

In [4]:
nasa_list = []
for result in nasa_results:
     # Error handling
    try:
        # Identify and return title of listing
        title = result.find('div', class_="content_title").text
#         # Identify and return price of listing
#         title = result.find('div', class_="content_title").text
#         # Identify and return link to listing
        text = result.find('div', class_="rollover_description_inner").text

        # Print results only if title, price, and link are available
        if (title and text):
#             print('-------------')
#             print(title)
#             print(text)

            nasa_list.append({"Nasa_Title":title.replace("\n",""),"Nasa_Text":text.replace("\n","")})

    except AttributeError as e:
        print(e)

In [5]:
#nasa_list

### Code for getting featured mars image from JPL site.

In [6]:
#setup for splinter
executable_path = {'executable_path': 'c:/bin/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [7]:
url = 'http://www.jpl.nasa.gov/spaceimages/?search=&category=Mars#submit/'
browser.visit(url)

In [8]:
html = browser.html
soup = BeautifulSoup(html, "html.parser")


In [9]:
mars_images = soup.find_all('li', class_='slide')

In [10]:
#print(mars_image)
i=0
mars_images_url = []
for mars_image in mars_images:
    image = mars_image.find('a')["data-fancybox-href"]
    image_link = "http://www.jpl.nasa.gov"+image
    print(image_link)
    mars_images_url.append(image_link)
    i+=1
    if i == 2:
        break

http://www.jpl.nasa.gov/spaceimages/images/largesize/PIA24088_hires.jpg
http://www.jpl.nasa.gov/spaceimages/images/largesize/PIA24087_hires.jpg


In [ ]:
mars_images_url

In [ ]:
browser.quit()

### Getting information table from Space-Facts

In [ ]:
#setup for splinter
executable_path = {'executable_path': 'c:/bin/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url = 'https://space-facts.com/mars/'
browser.visit(url)

In [ ]:
tables = pd.read_html(url)

In [ ]:
#tables is a list of dataframes.  Inspection found table[0] is desired one.

In [ ]:
mars_facts = tables[0].copy()
mars_facts.rename(columns = {0:"Parameter", 1:"Value"}, inplace=True) #renaming headings so make sense.
mars_facts

In [ ]:
mars_facts.to_html("mars_facts.html", index = False)

In [ ]:
browser.quit()

### Getting information from Astrogeology Site

In [ ]:
#setup for splinter
executable_path = {'executable_path': 'c:/bin/chromedriver.exe'}

browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
time.sleep(5) #added delay to make sure loads OK without issues

In [ ]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
hemi_list = ["Cerberus", "Schiaparelli", "Syrtis", "Valles"]
print(soup.prettify())

In [ ]:
base_url = "https://astrogeology.usgs.gov"
results = soup.find_all('div', class_='description')

hemisphere=[]
#print(results)
for result in results:
#     print('*************')
#     print(result)
#     print(type(result))
#     print('* ** ** ** ** ** *')
    
    try:
        test = result.find('a')["href"]
        name = result.find('h3').text
#         print(test)
#         print(name)
#        print("** ** ** **")
        # Access the href attribute with bracket notation
        #link = result.a['href']
        combine_url = base_url+test
        #print(combine_url)
        hemisphere.append({"title":name, "img_url":combine_url})
        
        
    except AttributeError as e:
        print(e)

browser.quit()

In [ ]:
print(hemisphere)

    

In [ ]:
print(f"NASA Title:  "+nasa_list[0]["Nasa_Title"])
print("***************")
print(f"NASA Story:  "+nasa_list[0]["Nasa_Text"])

In [ ]:
mars_images_url



In [ ]:
mars_facts

In [ ]:
title = hemisphere[0]["title"]
image_url = hemisphere[0]["img_url"]
print(f"Image Title:  {title}")
print(f"Image URL:  {image_url}")


In [ ]:
output_dictionnary = {}
output_dictionnary = {"Title":nasa_list[0]["Nasa_Title"],"Text":nasa_list[0]["Nasa_Text"]}
output_dictionnary["Featured_Image"] = mars_images_url[0]
output_dictionnary["Mars_Table"] = "mars_facts.html"
output_dictionnary.update({"Hemi_0":hemisphere[0]["title"],"Hemi_0_Img":hemisphere[0]["img_url"]})
output_dictionnary.update({"Hemi_1":hemisphere[1]["title"],"Hemi_0_Img":hemisphere[1]["img_url"]})
output_dictionnary.update({"Hemi_2":hemisphere[2]["title"],"Hemi_2_Img":hemisphere[2]["img_url"]})
output_dictionnary.update({"Hemi_3":hemisphere[3]["title"],"Hemi_3_Img":hemisphere[3]["img_url"]})




In [ ]:
print(output_dictionnary)